In [ ]:
import networkx as nx
import scipy as sp
import numpy as np
import scipy.sparse.linalg
import sys

# Eigenvector Centrality Bottom Counterexamples

In [ ]:
# Values in Table 2

kk = [8,9,10,11,12,14,15,16,17,18,19,24,26,27,28,29,30,31,32,34,35,36,37,38,40,43,44,45,48,50,51,56,57,59,61,62,63,64,65,66,66,68,69,70,72,73,74,75,76,77,78,80,82,84]
ss = [40,53,67,83,101,142,165,190,217,246,276,456,541,586,633,682,733,786,840,955,1015,1077,1141,1207,1344,1564,1641,1720,1968,2143,2233,2713,2815,3024,3241,3352,3465,3580,3697,3816,3816,4059,4184,4310,4569,4701,4835,4971,5109,5249,5391,5680,5977,6282]

In [ ]:
def eigenvector_centrality(G):
    A = nx.to_scipy_sparse_matrix(G);
    v = np.array([1.0] * G.number_of_nodes())
    while True:
        vlast = v
        v = vlast * A + vlast
        v /= np.max(v)
        # Check for convergence (in the supremum norm).
        if (np.absolute(v - vlast) < 1E-9).all():
            return v


In [ ]:
for i in range(len(kk)):
    k = kk[i]
    s = ss[i]
    
    # Graph in Figure 6
    
    G = nx.Graph()
    G.add_nodes_from(range(4 + k + k + s))

    # Three sides of the square
    G.add_edge(1,2)
    G.add_edge(2,3)
    G.add_edge(3,0)

    # Clique around 0: [4..4+k)
    for x in range(4, 4 + k):
        G.add_edge(0, x)
        for y in range(x + 1, 4 + k):
            G.add_edge(x, y)

    # Clique around 2: [4+k..4+k+k)
    for x in range(4 + k, 4 + k + k):
        G.add_edge(2, x)
        for y in range(x + 1, 4 + k + k):
            G.add_edge(x, y)

    # Star around 1: [4+k+k..4+k+k+s)
    for x in range(4 + k + k, 4 + k + k + s):
        G.add_edge(1, x)

    eig_pre = eigenvector_centrality(G)

    assert eig_pre[0] > eig_pre[1]
    # 1 is more important than 4 before adding the edge 0-1
    assert eig_pre[1] > eig_pre[4]
    
    G.add_edge(0, 1)
    eig_post = eigenvector_centrality(G)

    # 1 is less important than 4 after adding the edge 0-1
    assert eig_post[1] < eig_post[4]
    # The rank of 1 has decreased
    assert sum([1 if eig_pre[x] > eig_pre[1] else 0 for x in range(len(eig_pre))]) < sum([1 if eig_post[x] > eig_post[1] else 0 for x in range(len(eig_post))])

    print(k, s, "OK")
    sys.stdout.flush()
